In [ ]:
import cv2
import numpy as np

# 创建一张A4纸大小的空白图像，通常使用300 DPI分辨率
width, height = int(21000), int(29700)  # 将英寸转换为像素
a4_paper = np.ones((height, width, 3), np.uint8) * 255  # 创建一个白色背景图像

# 导入自定义大小的图像并调整大小以适应A4纸
image = cv2.imread('my_plot.png')  # 替换为您的图像文件路径

# 计算目标尺寸
target_width = int(1673)
target_height = int(992)

# 检查目标尺寸是否合法
if target_width > 0 and target_height > 0:
    image = cv2.resize(image, (target_width, target_height))

    # 在A4背景图像上绘制导入的图像，放置在A4中央
    x_offset = (a4_paper.shape[1] - image.shape[1]) // 2
    y_offset = (a4_paper.shape[0] - image.shape[0]) // 2
    a4_paper[y_offset:y_offset + image.shape[0], x_offset:x_offset + image.shape[1]] = image

    
    # 保存生成的图像
    cv2.imwrite('output_image.png', a4_paper)
else:
    print("目标尺寸无效")

只用opencv，在A4纸上生成所需的扇环

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# A4
width, height = int(21000), int(29700)
image = np.ones((height, width, 3), dtype=np.uint8)*255

# 定义扇形的参数
center = (5000, 0)  # 扇形的中心坐标
in_radius = 3082 #内圆半径
radius = 4206  # 扇形的半径
start_angle = 0  # 扇形的起始角度（度数）
end_angle = 163.49  # 扇形的终止角度（度数）
color = (0, 255, 0)  # 扇形的颜色，这里是绿色
inner_color = (255,255,255)
thickness = -1  # 填充扇形，使用负数线宽值

# 使用OpenCV的绘图函数绘制扇形
cv2.ellipse(image, center, (radius, radius), 0, start_angle, end_angle, color, thickness)
cv2.ellipse(image, center, (in_radius, in_radius), 0, start_angle, end_angle, inner_color, thickness)


# 保存绘制好的图像
cv2.imwrite('fanRing.png', image)
# plt.imshow(image)

# 显示图像
# cv2.imshow('fanRing', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


生成一圈朝向圆心的二维码

In [ ]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import os


# 定义图片文件夹路径
image_folder = '/home/daic/Desktop/cv_work/output_markers'

# 获取图片文件夹中的所有文件
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

# 创建一个空列表来存储读取的图片
images = []

# 使用循环读取每张图片并存储在列表中
for image_file in image_files:
    print(image_file)
    image = cv2.imread(image_file)
    if image is not None:
        images.append(image)
    else:
        print(f"Failed to read image: {image_file}")


# 图片尺寸和数量
image_width, image_height = 1000, 1000
num_images = len(images)

# 圆环的半径和中心坐标
radius = 2000
center_x, center_y = 3000, 3000


# 创建一个空白画布
canvas = 255 * np.ones((6000, 6000, 3), dtype=np.uint8)

# 计算每张图片的角度间隔
angle_step = 360 / num_images


for i in range(num_images):
    print(f'i={i}')
    angle_deg = i * angle_step
    angle_rad = math.radians(angle_deg) #angle_rad = angle_deg * π / 180
    x = int(center_x + radius * math.cos(angle_rad)) - image_width // 2
    y = int(center_y + radius * math.sin(angle_rad)) - image_height // 2


    # 计算旋转角度，使图片面向圆心
    rotation_angle_deg = -angle_deg  # 负号用于逆时针旋转
    rotation_matrix = cv2.getRotationMatrix2D((image_width // 2, image_height // 2), rotation_angle_deg, 1)
    rotated_image = cv2.warpAffine(images[i], rotation_matrix, (image_width, image_height))

    # 将旋转后图片放在计算出的位置上
    canvas[y:y + image_height, x:x + image_width] = rotated_image

    plt.imshow(canvas)
# # 显示圆环
# cv2.imshow('Circular Layout with Rotation', canvas)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

旋转图片不截断

In [ ]:
import cv2
import numpy as np

# 读取图片
image = cv2.imread('westbrook.jpg')

# 获取图片的中心点坐标
height, width = image.shape[:2]
center = (width // 2, height // 2)

# 定义旋转矩阵
angle = 45  # 旋转角度
rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

# 计算旋转后的图像尺寸
cosine = np.abs(rotation_matrix[0, 0])
sine = np.abs(rotation_matrix[0, 1])
new_width = int((height * sine) + (width * cosine))
new_height = int((height * cosine) + (width * sine))

# 调整旋转矩阵以确保图像不被裁剪
rotation_matrix[0, 2] += (new_width / 2) - center[0]
rotation_matrix[1, 2] += (new_height / 2) - center[1]

# 进行旋转
rotated_image = cv2.warpAffine(image, rotation_matrix, (new_width, new_height), flags=cv2.INTER_LINEAR)

# 显示旋转后的图像
cv2.imshow('Rotated Image', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


完整生成带二维码的扇环61-68

In [61]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import os


# A4
width, height = int(21000), int(29700)
canvas = np.ones((height, width, 3), dtype=np.uint8)*255

# 定义扇环的参数
center = (5000, 1000)  # 扇形的中心坐标

# 定义图片文件夹路径
image_folder = 'E:\OneDrive\FineTool\cv-work\cv_work\output_6168'

# 获取图片文件夹中的所有文件
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

# 创建一个空列表来存储读取的图片
images = []

# 使用循环读取每张图片并存储在列表中
for image_file in image_files:
    print(image_file)
    image = cv2.imread(image_file)
    if image is not None:
        images.append(image)
    else:
        print(f"Failed to read image: {image_file}")


# aruco图片尺寸和数量
image_width, image_height = 800, 800
center_aruco = (image_width//2,image_height//2)
num_images = len(images)

# 二维码环的半径
radius = 3644 #in_radius + radius /2

# 计算每张图片的角度间隔
angle_step = 150 / num_images

points = []
for i in range(num_images):
    print(f'i={i}')
    angle_deg = (i+1)* angle_step
    angle_rad = math.radians(angle_deg) #angle_rad = angle_deg * π / 180
    # x = int(center_x + radius * math.cos(angle_rad)) - image_width // 2
    # y = int(center_y + radius * math.sin(angle_rad)) - image_height // 2
    x = int(center[0] + radius * math.cos(angle_rad)) 
    y = int(center[1] + radius * math.sin(angle_rad)) 
    points.append((x,y))


    images[i] = cv2.resize(images[i],(image_width,image_height))

    # 计算旋转角度，使图片面向圆心
    rotation_angle_deg = -angle_deg  # 负号用于逆时针旋转
    rotation_matrix = cv2.getRotationMatrix2D((image_width // 2, image_height // 2), rotation_angle_deg, 1)
    #计算旋转后的尺寸
    cosine = np.abs(rotation_matrix[0,0])
    sine = np.abs(rotation_matrix[0,1])
    new_width = int((image_height*sine)+(image_width*cosine))
    new_height = int((image_height*cosine)+(image_width*sine))

    #调整旋转矩阵确保图像不被裁剪
    rotation_matrix[0,2] += (new_width/2) - center_aruco[0]
    rotation_matrix[1,2] += (new_height/2) - center_aruco[1]

    #进行旋转
    rotated_image = cv2.warpAffine(images[i], rotation_matrix, (new_width, new_height),flags=cv2.INTER_LINEAR ,borderValue=(255, 255, 255))

    # 将旋转后图片放在计算出的位置上
    # canvas[y:y + new_height, x:x + new_width] = rotated_image
    canvas[int(y-new_height/2):int(y + new_height/2), int(x-new_width/2):int(x + new_width/2)] = rotated_image

#绘制扇环

in_radius = 3082 #内圆半径
radius = 4206  # 扇形的半径
start_angle = 0  # 扇形的起始角度（度数）
end_angle = 163.49  # 扇形的终止角度（度数）
color = (0, 0, 0)  # 外扇环的颜色，
inner_color = (0,0,0)
thickness = 10  # 填充扇形，使用负数线宽值

# 使用OpenCV的绘图函数绘制扇形
cv2.ellipse(canvas, center, (radius, radius), 0, start_angle, end_angle, color, thickness);
cv2.ellipse(canvas, center, (in_radius, in_radius), 0, start_angle, end_angle, inner_color, thickness);

#画一个10X10的标准框用于测量
top_left = (500,1000)
bottom_right = (10500,6000)
cv2.rectangle(canvas,top_left,bottom_right,color,thickness)

cv2.imwrite('arucoRing.png', canvas)




E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_1.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_2.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_3.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_4.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_5.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_6.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_7.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_8.png
i=0
i=1
i=2
i=3
i=4
i=5
i=6
i=7


True

定义两个函数
函数def generateMyArucoMarkers(output_folder,start_ID,marker_nums):
生成指定文件下连续ID号的DICT_4X4_250二维码函数，大小1000X1000  
输入参数

函数def generateArucoSetor(image_folder,center,image_width,image_height,radius):
生成指定文件夹下扇环形状的二维码，均匀排列
输入参数：文件名，中心点，二维码宽度，二维码高度，二维码围成的半径

In [8]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2.aruco as aruco


def generateMyArucoMarkers(output_folder,start_ID,marker_nums):
# 创建目标文件夹
    os.makedirs(output_folder, exist_ok=True)
    markers = []
    image_size = 1000
    marker_size = 1000
    marker_nums = 8

    markerImage = np.zeros((image_size,image_size), np.uint8)
    #markerImage = cv.cvtColor(blank_markerImage,cv.COLOR_BGR2GRAY)
    dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    for i in range(marker_nums):
        output_markername = os.path.join(output_folder,f'marker_{i+1}.png')
        generator = aruco.generateImageMarker(dictionary,int(i+start_ID),marker_size,markerImage,1)

        success = cv2.imwrite(output_markername,generator)
        if success:
            print(f'Saved {output_markername}')
        else:
            print(f'Failed to save {output_markername}')
        markers.append(generator)




def generateArucoSetor(image_folder,center,image_width,image_height,radius,angle):
   # 获取图片文件夹中的所有文件
    image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

    # 创建一个空列表来存储读取的图片
    images = []

    # 使用循环读取每张图片并存储在列表中
    for image_file in image_files:
        print(image_file)
        image = cv2.imread(image_file)
        if image is not None:
            images.append(image)
        else:
            print(f"Failed to read image: {image_file}")

    center_aruco = (image_width//2,image_height//2)
    num_images = len(images)

    # 计算每张图片的角度间隔
    angle_step = angle / num_images

    points = []
    for i in range(num_images):
        print(f'i={i}')
        angle_deg = (i)* angle_step + 10
        angle_rad = math.radians(angle_deg) #angle_rad = angle_deg * π / 180
        # x = int(center_x + radius * math.cos(angle_rad)) - image_width // 2
        # y = int(center_y + radius * math.sin(angle_rad)) - image_height // 2
        x = int(center[0] + radius * math.cos(angle_rad)) 
        y = int(center[1] + radius * math.sin(angle_rad)) 
        points.append((x,y))


        images[i] = cv2.resize(images[i],(image_width,image_height))

        # 计算旋转角度，使图片面向圆心
        rotation_angle_deg = -angle_deg  # 负号用于逆时针旋转
        rotation_matrix = cv2.getRotationMatrix2D((image_width // 2, image_height // 2), rotation_angle_deg, 1)
        #计算旋转后的尺寸
        cosine = np.abs(rotation_matrix[0,0])
        sine = np.abs(rotation_matrix[0,1])
        new_width = int((image_height*sine)+(image_width*cosine))
        new_height = int((image_height*cosine)+(image_width*sine))

        #调整旋转矩阵确保图像不被裁剪
        rotation_matrix[0,2] += (new_width/2) - center_aruco[0]
        rotation_matrix[1,2] += (new_height/2) - center_aruco[1]

        #进行旋转
        rotated_image = cv2.warpAffine(images[i], rotation_matrix, (new_width, new_height),flags=cv2.INTER_LINEAR ,borderValue=(255, 255, 255))

        # 将旋转后图片放在计算出的位置上
        # canvas[y:y + new_height, x:x + new_width] = rotated_image
        canvas[int(y-new_height/2):int(y + new_height/2), int(x-new_width/2):int(x + new_width/2)] = rotated_image

用函数生成扇环二维码（老版）

In [4]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import os

# A4
width, height = int(21000), int(29700)
canvas = np.ones((height, width, 3), dtype=np.uint8)*255

#################输入参数6168########################
# 定义扇环的参数
center = (5000, 1000)  # 扇形的中心坐标
# 定义图片文件夹路径
image_folder = 'E:\OneDrive\FineTool\cv-work\cv_work\output_6168'
# aruco图片尺寸
image_width, image_height = 800, 800
# 二维码环的半径
radius = 3644 #in_radius + radius /2
#扇环张角
angel = 163.49

#################输入参数6976########################
# 定义扇环的参数
center2 = (5000, 7000)  # 扇形的中心坐标
# 定义图片文件夹路径
image_folder2 = 'E:\OneDrive\FineTool\cv-work\cv_work\output_6976'
# aruco图片尺寸
# image_width, image_height = 800, 800
# 二维码环的半径
# radius = 3644 #in_radius + radius /2

generateArucoSetor(image_folder,center,image_width,image_height,radius,angel)
generateArucoSetor(image_folder2,center2,image_width,image_height,radius,angel)

#绘制扇环

in_radius = 3082 #内圆半径
radius = 4206  # 外圆的半径
start_angle = 0  # 扇形的起始角度（度数）
end_angle = 163.49  # 扇形的终止角度（度数）
color = (0, 0, 0)  # 外扇环的颜色，
inner_color = (0,0,0)
thickness = 10  # 填充扇形，使用负数线宽值

# 使用OpenCV的绘图函数绘制扇形
cv2.ellipse(canvas, center, (radius, radius), 0, start_angle, end_angle, color, thickness);
cv2.ellipse(canvas, center, (in_radius, in_radius), 0, start_angle, end_angle, inner_color, thickness);

cv2.ellipse(canvas, center2, (radius, radius), 0, start_angle, end_angle, color, thickness);
cv2.ellipse(canvas, center2, (in_radius, in_radius), 0, start_angle, end_angle, inner_color, thickness);

#画一个10X10的标准框用于测量
top_left = (500,1000)
bottom_right = (10500,6000)
cv2.rectangle(canvas,top_left,bottom_right,color,thickness)

top_left = (500,7000)
bottom_right = (10500,12000)
cv2.rectangle(canvas,top_left,bottom_right,color,thickness)

cv2.imwrite('arucoRing.png', canvas)




E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_1.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_2.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_3.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_4.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_5.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_6.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_7.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_8.png
i=0
i=1
i=2
i=3
i=4
i=5
i=6
i=7
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_1.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_2.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_3.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_4.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_5.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_6.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_7.png
E:\OneDrive\FineTool\cv-work\cv_work\o

True

用函数生成新版扇环二维码

In [9]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import os

# A4
width, height = int(21000), int(29700)
canvas = np.ones((height, width, 3), dtype=np.uint8)*255

#################输入参数6168########################
# 定义扇环的参数
center = (5000, 1000)  # 扇形的中心坐标
# 定义图片文件夹路径
image_folder = 'E:\OneDrive\FineTool\cv-work\cv_work\output_6168'
# aruco图片尺寸
image_width, image_height = 800, 800
# 二维码环的半径
radius = 4019.5 #in_radius + radius /2
#扇环张角
angel = 150.449


#################输入参数6976########################
# 定义扇环的参数
center2 = (5000, 7000)  # 扇形的中心坐标
# 定义图片文件夹路径
image_folder2 = 'E:\OneDrive\FineTool\cv-work\cv_work\output_6976'
# aruco图片尺寸
# image_width, image_height = 800, 800
# 二维码环的半径
# radius = 3644 #in_radius + radius /2

generateArucoSetor(image_folder,center,image_width,image_height,radius,angel)
generateArucoSetor(image_folder2,center2,image_width,image_height,radius,angel)

#绘制扇环

in_radius = 3469 #内圆半径
radius = 4570  # 外圆的半径
start_angle = 0  # 扇形的起始角度（度数）
end_angle = 150.449  # 扇形的终止角度（度数）
color = (0, 0, 0)  # 外扇环的颜色，
inner_color = (0,0,0)
thickness = 10  # 填充扇形，使用负数线宽值

# 使用OpenCV的绘图函数绘制扇形
cv2.ellipse(canvas, center, (radius, radius), 0, start_angle, end_angle, color, thickness);
cv2.ellipse(canvas, center, (in_radius, in_radius), 0, start_angle, end_angle, inner_color, thickness);

cv2.ellipse(canvas, center2, (radius, radius), 0, start_angle, end_angle, color, thickness);
cv2.ellipse(canvas, center2, (in_radius, in_radius), 0, start_angle, end_angle, inner_color, thickness);

#画一个10X10的标准框用于测量
top_left = (500,1000)
bottom_right = (10500,6000)
cv2.rectangle(canvas,top_left,bottom_right,color,thickness)

top_left = (500,7000)
bottom_right = (10500,12000)
cv2.rectangle(canvas,top_left,bottom_right,color,thickness)

cv2.imwrite('arucoRingNew.png', canvas)




E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_1.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_2.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_3.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_4.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_5.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_6.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_7.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6168\marker_8.png
i=0
i=1
i=2
i=3
i=4
i=5
i=6
i=7
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_1.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_2.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_3.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_4.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_5.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_6.png
E:\OneDrive\FineTool\cv-work\cv_work\output_6976\marker_7.png
E:\OneDrive\FineTool\cv-work\cv_work\o

True

In [36]:
import cv2
import numpy as np
import math

# 已知圆的信息
center = (200, 200)  # 圆心坐标 (x, y)
radius = 100  # 半径

# 创建一个空白图像
image = np.zeros((400, 400, 3), dtype=np.uint8)

# 计算8个点的坐标
points = []
for i in range(8):
    angle = 2 * math.pi * i / 8  # 在圆上等间距分布的角度
    x = int(center[0] + radius * math.cos(angle))
    y = int(center[1] + radius * math.sin(angle))
    points.append((x, y))

# 在图像上绘制这些点
for point in points:
    cv2.circle(image, point, 5, (0, 0, 255), -1)  # 绘制红色点

# 显示图像
cv2.imshow('Circle with 8 Equally Spaced Points', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [56]:
import cv2
import numpy as np

# 创建一个空白图像
canvas = np.zeros((400, 400, 3), dtype=np.uint8)

# 定义椭圆参数
center = (200, 200)  # 圆心坐标 (x, y)
outer_radius = 100  # 外部半径
inner_radius = 80   # 内部半径
start_angle = 30    # 起始角度
end_angle = 330     # 结束角度
color_outer = (0, 0, 255)  # 外部轮廓颜色
color_inner = (0, 255, 0)  # 内部轮廓颜色
thickness = 2       # 边框宽度

# 绘制外部轮廓
cv2.ellipse(canvas, center, (outer_radius, outer_radius), 0, start_angle, end_angle, color_outer, thickness)

# 绘制内部轮廓
cv2.ellipse(canvas, center, (inner_radius, inner_radius), 0, start_angle, end_angle, color_inner, thickness)

# 显示图像
cv2.imshow('Sector with Inner and Outer Contours', canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import math

# 已知直角边的长度
a = float(input("请输入第一个直角边的长度："))
b = float(input("请输入第二个直角边的长度："))

# 使用勾股定理计算斜边的长度
c = math.sqrt(a**2 + b**2)

# 输出结果
print(f"斜边的长度为：{c}")


斜边的长度为：1.100727032465361


In [6]:
import math

# 已知圆的半径
radius = float(input("请输入圆的半径："))

# 计算圆的周长
circumference = 2 * math.pi * radius

# 输出结果
print(f"圆的周长为：{circumference}")

圆的周长为：9.1106186954104
